In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.dask

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn import metrics

import scipy

import dask
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, SSHCluster
import dask.distributed

import dask_ml
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import LabelEncoder

import xgboost as xgb

import mpl_scatter_density
import matplotlib.colors as colors

import csv

## Setting up the cluster

In [3]:
#cluster.close()

In [2]:
cluster = SSHCluster(["bhbh-1", "bhbh-1", "bhbh-2", "bhbh-3", "bhbh-4", "bhbh-5"],
                    connect_options={"client_keys": "/home/ubuntu/private/tbertola_key.pem"},
                    worker_options={"n_workers": 4,
                                    "nthreads": 1}, # because each bhbh-* has 4 cores
                    scheduler_options={"port": 8786, "dashboard_address": ":8787"}
                    )

2023-06-28 16:28:27,586 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:27,585 - distributed.scheduler - INFO - State start
2023-06-28 16:28:27,599 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:27,598 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.140:8786
2023-06-28 16:28:28,608 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:28,607 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:42547'
2023-06-28 16:28:28,624 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:28,623 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:46595'
2023-06-28 16:28:28,628 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:28,627 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:33083'
2023-06-28 16:28:28,631 - distributed.deploy.ssh - INFO - 2023-06-28 16:28:28,631 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:35285'
2023-06-28 16:28:28,709 - distributed.deploy.ssh - INFO - 2023-06-28

In [3]:
client=Client(cluster)

/home/ubuntu/miniconda3/lib/python3.10/site-packages/distributed/client.py:1388: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+------------------+
| Package | Client | Scheduler | Workers          |
+---------+--------+-----------+------------------+
| tornado | 6.3.2  | 6.3.2     | {'6.3.2', '6.2'} |
+---------+--------+-----------+------------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
client

<Client: 'tcp://10.67.22.140:8786' processes=20 threads=20, memory=38.78 GiB>

## Data

In [5]:
#defin the value of q to drop
q_drop = 0.2

#import data
bhbh = dd.read_parquet('/home/ubuntu/data/bag_all_dataset_with_no_kick/*')

#Masking on the data
bhbh = bhbh[bhbh['q'] > q_drop] #dropping the value of q under q_drop
#bhbh = bhbh[ bhbh['No_Kick'] == 2] #taking only the binaries without kick

## Discretizing and introducing labels

In [6]:
#add column label
n_bins = 100
# Define the edges between bins
counts, bin_edges = da.histogram(bhbh.q.values, bins=n_bins, range=(q_drop,1.0))

# pd.cut each column, with each bin closed on left and open on right
bhbh['label'] = bhbh['q'].map_partitions(pd.cut, bin_edges, right=True, labels=False, include_lowest=False)

## Splitting the data in train and set

In [7]:
bhbh_train, bhbh_test = train_test_split(bhbh, train_size=0.7, random_state=1, shuffle=True)

bhbh_train = bhbh_train.reset_index(drop=True) # just for simplicity sake in the final visualisation
bhbh_test = bhbh_test.reset_index(drop=True)   # and so that we don't mess up later when checking the results

In [11]:
bhbh_train.to_parquet('/mnt/bhbh/train')
bhbh_test.to_parquet('/mnt/bhbh/test/')

In [12]:
cluster.close()